In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact, IntSlider

def apply_action(grid, action, max_x, max_y, target):
    """Applique une action RECT ou JOKER sur la grille."""
    parts = action.split()
    if parts[0] == "RECT":
        _, x1, y1, x2, y2, color = parts
        x1, y1, x2, y2, color = map(int, (x1, y1, x2, y2, color))
        grid[y1:y2+1, x1:x2+1] = color
    elif parts[0] == "JOKER":
        _, x1, y1, x2, y2 = parts
        x1, y1, x2, y2 = map(int, (x1, y1, x2, y2))
        grid[y1:y2+1, x1:x2+1] = target[y1:y2+1, x1:x2+1]
    return grid

def compute_score(target, current, step, max_actions):
    """Calcule le score à une étape donnée."""
    total_pixels = target.size
    correct_pixels = np.sum(target == current)
    
    if correct_pixels < total_pixels:
        return round(1_000_000 * correct_pixels / total_pixels)
    else:
        return round(1_000_000 * max_actions / step)

def viewer(input_file, solution_file):
    # Charger le dataset
    with open(input_file, "r") as f:
        data = json.load(f)
    target = np.array(data["grid"])
    max_actions = data["maxActions"]

    # Charger la solution
    with open(solution_file, "r") as f:
        actions = [line.strip() for line in f if line.strip()]
    
    # Préparer états successifs
    h, w = target.shape
    states = [np.zeros_like(target)]
    for action in actions:
        new_state = states[-1].copy()
        new_state = apply_action(new_state, action, w, h, target)
        states.append(new_state)

    def show(step=0):
        pixel_colormap = matplotlib.colors.ListedColormap(['#000000', '#FFFFFF', '#1E93FF', '#F93C31', '#4FCC30', '#FFDC00', '#E53AA3', '#FF851B'])

        fig, axes = plt.subplots(1, 2, figsize=(10, 5))
        # Fresque cible
        axes[0].imshow(target, cmap=pixel_colormap, vmin=0, vmax=7)
        axes[0].set_title("Fresque cible")
        
        
        # Fresque produite
        axes[1].imshow(states[step], cmap=pixel_colormap, vmin=0, vmax=7)
        axes[1].set_title(f"Solution - étape {step}")
        
        for ax in axes:
            # Cadres noirs autour des images
            ax.axis("on")
            for spine in ax.spines.values():
                spine.set_edgecolor('black')
                spine.set_linewidth(2)
            ax.set_xticks([])
            ax.set_yticks([])

        # Score
        score = compute_score(target, states[step], max(1, step), max_actions)
        plt.suptitle(f"Score à l'étape {step}: {score}", fontsize=14)
        plt.show()

    interact(show, step=IntSlider(min=0, max=len(actions), step=1, value=0))

In [ ]:
viewer("./datasets/1_example.json", "./solutions/1_example_submission.txt")
